In [3]:
import pickle
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as keras_image
import gradio as gr
import os

In [4]:
loaded_model = load_model("model_epoch_06.h5", compile=False)
model_config = loaded_model.get_config()
#loaded_model.summary()

In [7]:
def pred(img):

    # Resize the image to match the input shape of your model
    img = img.resize((256, 256))  # Adjust the size as needed
    # Convert the image to a NumPy array
    img_np = np.array(img)
    
    # Preprocess the image to match the model's input requirements
    img_np = keras_image.img_to_array(img_np)
    
    img_for_plot = img_np / 255.0  # Normalize the image if necessary
    
    img_np = np.expand_dims(img_np, axis=0)  # Add a batch dimension
    
    # Make predictions
    predictions = loaded_model.predict(img_np)
    return img_for_plot, predictions

def image_mod(img):
    
    image, predictions = pred(img)

    print(predictions)
    
    result = ""
    
    if predictions[0][0] == 0.0:

        result = "fake image"
    else :
        
        result = "true image"

    return result

In [ ]:
title = """<h1 align="center">🔥AI臉部辨識🚀</h1>"""
                
with gr.Blocks(css = """.gradio-container {background-color: #3f7791}""") as demo:
    gr.HTML(title)
    gr.HTML('''<center><a href="https://github.com/kennywang112?tab=repositories" alt="GitHub Repo"></a></center>''')
    with gr.Row():
        with gr.Column(elem_id = "col_container"):
            chatbot = gr.Chatbot(elem_id='chatbot')
            inputs = gr.Textbox(placeholder= "Hi there!", label= "Type an input and press Enter")
            state = gr.State([])
            b1 = gr.Button()

        with gr.Column(elem_id = "col_container"):
            gr.Interface(
                image_mod,
                gr.Image(type="pil"),
                "text",
                examples=[
                    os.path.join("images/real_face0.jpeg"),
                    os.path.join("images/real_face1.jpeg"),
                    os.path.join("images/fake_face1.jpeg"),
                    os.path.join("images/fake_face0.jpeg"),
                ],
            )

        # inputs.submit( predict, [inputs, top_p, temperature, openai_api_key, chat_counter, chatbot, state], [chatbot, state, chat_counter],)
        # b1.click( predict, [inputs, top_p, temperature, openai_api_key, chat_counter, chatbot, state], [chatbot, state, chat_counter],)
        # b1.click(reset_textbox, [], [inputs])
        # inputs.submit(reset_textbox, [], [inputs])

        #gr.Markdown(description)
demo.queue().launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 209ms/step
[[0. 1.]]
